In [ ]:
import os
import warnings
import torch
import numpy as np
import torchvision.transforms as transforms
from PIL import Image
import segmentation_models_pytorch as smp
#pip install git+https://github.com/qubvel/segmentation_models.pytorch

# Ignore warnings
warnings.filterwarnings('ignore')

# Paths to images and models
image_path = "D:/sumon/2d_segment/content/Data _foot/Test/fold_1/images/0828.PNG"
save_root = "D:/sumon/2d_segment/save_masks"

# Model files and corresponding save directories
models_info = {
    "efficientnet-b3": {
        "pt_file": "D:/sumon/2d_segment/content/ResultsFootUlcer/efficientnet-b3_SelfONN_FPN/efficientnet-b3_SelfONN_FPN_fold_1.pt",
        "save_folder": os.path.join(save_root, "masks_b3"),
    },
    "efficientnet-b4": {
        "pt_file": "D:/sumon/2d_segment/content/ResultsFootUlcer/efficientnet-b4_SelfONN_FPN/efficientnet-b4_SelfONN_FPN_fold_1.pt",
        "save_folder": os.path.join(save_root, "masks_b4"),
    },
    "efficientnet-b5": {
        "pt_file": "D:/sumon/2d_segment/content/ResultsFootUlcer/efficientnet-b5_SelfONN_FPN/efficientnet-b5_SelfONN_FPN_fold_1.pt",
        "save_folder": os.path.join(save_root, "masks_b5"),
    }
}

# Ensure all save directories exist
for model_name, info in models_info.items():
    os.makedirs(info["save_folder"], exist_ok=True)

# Load the grayscale image
image = Image.open(image_path).convert('L')  # Ensure grayscale mode

# Convert to NumPy array and normalize
gray_image = np.array(image, dtype=np.float32) / 255.0  # Normalize pixel values

# Define transformation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.40936773], std=[0.33767385])
])

# Convert to tensor and add batch dimension
input_tensor = transform(image).unsqueeze(0)  # Shape: [1, 1, 512, 512]

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_tensor = input_tensor.to(device)

# Iterate through models
for model_name, info in models_info.items():
    print(f"Processing {model_name}...")

    # Load checkpoint
    checkpoint = torch.load(info["pt_file"], map_location=device)

    # Define the model architecture correctly
    # model = smp.FPN(encoder_name=model_name, classes=1, activation=None)
    model = checkpoint['model']
    # Load model weights
    # if 'model' in checkpoint:
    #     model.load_state_dict(checkpoint['model'])
    # else:
    #     model.load_state_dict(checkpoint)

    model = model.to(device)
    model.eval()  # Set to evaluation mode

    # Run inference
    with torch.no_grad():
        output = model(input_tensor)  # Shape: [1, 1, 512, 512]
        output_cpu = torch.sigmoid(output).squeeze().cpu().numpy()  # Apply sigmoid activation

    # Apply threshold to create a binary mask
    threshold = 0.5
    binary_mask = (output_cpu > threshold).astype(np.uint8) * 255  # Convert to 0-255

    # Convert NumPy array to grayscale PIL Image
    binary_mask_img = Image.fromarray(binary_mask)  # Ensure grayscale

    # Extract image name without extension
    img_name = os.path.basename(image_path).split('.')[0]

    # Save the mask
    save_path = os.path.join(info["save_folder"], f"{img_name}_mask.png")
    binary_mask_img.save(save_path)

    print(f"Saved mask for {model_name} at: {save_path}")

print( "All masks saved successfully!")


C:\Users\QUMLG\.conda\envs\multi\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Processing efficientnet-b3...
Saved mask for efficientnet-b3 at: D:/sumon/2d_segment/save_masks\masks_b3\0828_mask.png
Processing efficientnet-b4...
Saved mask for efficientnet-b4 at: D:/sumon/2d_segment/save_masks\masks_b4\0828_mask.png
Processing efficientnet-b5...
Saved mask for efficientnet-b5 at: D:/sumon/2d_segment/save_masks\masks_b5\0828_mask.png
All masks saved successfully!


In [3]:
import SimpleITK as sitk
import os
import numpy as np
from tqdm import tqdm

def calculate_staple_consensus(mask_paths):
    """
    Calculate the STAPLE consensus for binary masks.

    Parameters:
    - mask_paths: List of paths to the binary mask images.

    Returns:
    - consensus_mask: Binary image representing the STAPLE consensus.
    """

    # Read the masks and convert them to binary (0 and 1)
    masks = [sitk.ReadImage(mask_path, sitk.sitkUInt8) for mask_path in mask_paths]
    
    # Normalize masks from [0, 255] to [0, 1] (STAPLE requires binary input)
    masks = [sitk.Cast(m > 127, sitk.sitkUInt8) for m in masks]  # Any value >127 becomes 1

    # Calculate the STAPLE consensus
    staple_filter = sitk.STAPLEImageFilter()
    consensus_mask = staple_filter.Execute(masks)

    return consensus_mask

# Define the output directory
out_dir = r'D:\sumon\2d_segment\save_masks\combined_mask'
os.makedirs(out_dir, exist_ok=True)

# Define the folders containing masks
mask_folders = [
    r'D:\sumon\2d_segment\save_masks\masks_b3',
    r'D:\sumon\2d_segment\save_masks\masks_b4',
    r'D:\sumon\2d_segment\save_masks\masks_b5'
]

# Choose a reference folder (for matching filenames)
reference_folder = mask_folders[0]

# Process each image in the reference folder
for image_filename in tqdm(os.listdir(reference_folder), desc='Processing STAPLE consensus'):
    mask_paths = [os.path.join(folder, image_filename) for folder in mask_folders]

    # Ensure all mask files exist
    if not all(os.path.exists(mask_path) for mask_path in mask_paths):
        print(f"Skipping {image_filename} as not all masks are present.")
        continue

    # Compute STAPLE consensus
    consensus = calculate_staple_consensus(mask_paths)

    # Convert STAPLE output to numpy array and check the value range
    consensus_array = sitk.GetArrayFromImage(consensus)
    print(f"Consensus range for {image_filename}: min={consensus_array.min()}, max={consensus_array.max()}")

    # Apply threshold to create a binary mask
    threshold_filter = sitk.BinaryThresholdImageFilter()
    threshold_filter.SetLowerThreshold(0.3)  # Adjust threshold if needed
    threshold_filter.SetUpperThreshold(1.0)
    threshold_filter.SetInsideValue(1)
    threshold_filter.SetOutsideValue(0)
    binary_consensus = threshold_filter.Execute(consensus)

    # Convert binary mask back to 0-255 range for saving
    binary_consensus = sitk.Cast(binary_consensus * 255, sitk.sitkUInt8)

    # Save the consensus mask
    save_path = os.path.join(out_dir, image_filename)
    sitk.WriteImage(binary_consensus, save_path)

print("✅ STAPLE consensus masks saved successfully!")


Processing STAPLE consensus:  50%|█████████         | 1/2 [00:00<00:00,  5.15it/s]

Consensus range for 0828_mask.png: min=0.0, max=1.0


Processing STAPLE consensus: 100%|█████████████████| 2/2 [17:19<00:00, 519.73s/it]

Consensus range for 0855_mask.png: min=0.0, max=0.36023546608267554
✅ STAPLE consensus masks saved successfully!
